In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

In [ ]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 50

# data generators
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2,
    brightness_range=[0.8, 1.2],
    zoom_range=0.2, 
    rotation_range=15,
    width_shift_range=0.2, 
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True
)


# Load training and validation datasets
train_data = train_datagen.flow_from_directory(
    "/content/drive/MyDrive/dataset/train",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="training"
)

val_data = train_datagen.flow_from_directory(
    "/content/drive/MyDrive/dataset/val",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="validation"
)

# indices
print("Class Indices (Train):", train_data.class_indices)
print("Class Indices (Validation):", val_data.class_indices)

Found 558 images belonging to 2 classes.
Found 138 images belonging to 2 classes.
Class Indices (Train): {'KTP': 0, 'non_KTP': 1}
Class Indices (Validation): {'KTP': 0, 'non_KTP': 1}


In [5]:
# Load pre-trained MobileNetV2
base_model = MobileNetV2(
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3),
    include_top=False,
    weights="imagenet"
)
base_model.trainable = False  # Freeze base model

9406464/9406464 [==============================] - 0s 0us/step


In [6]:
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),  # 30% dropout
    layers.Dense(1, activation="sigmoid") #binary
])


# Compile the model with additional metrics and learning rate schedule
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=10000,
    decay_rate=0.9
)

In [7]:
base_model.trainable = True  # Unfreeze the base model

# Fine-tune from a specific layer
for layer in base_model.layers[:100]:  # Freeze initial 100 layers
    layer.trainable = False

# Recompile with a lower learning rate for fine-tuning
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)


In [8]:
# early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

In [9]:
# Train model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS,
    callbacks=[early_stopping]
)

Epoch 1/50
18/18 [==============================] - 98s 5s/step - loss: 0.3913 - accuracy: 0.8405 - precision: 0.8393 - recall: 0.8423 - val_loss: 0.3140 - val_accuracy: 0.8841 - val_precision: 0.8732 - val_recall: 0.8986
Epoch 2/50
18/18 [==============================] - 14s 796ms/step - loss: 0.1235 - accuracy: 0.9606 - precision: 0.9509 - recall: 0.9713 - val_loss: 0.1980 - val_accuracy: 0.9348 - val_precision: 0.9688 - val_recall: 0.8986
Epoch 3/50
18/18 [==============================] - 14s 796ms/step - loss: 0.0765 - accuracy: 0.9785 - precision: 0.9751 - recall: 0.9821 - val_loss: 0.1468 - val_accuracy: 0.9348 - val_precision: 0.9054 - val_recall: 0.9710
Epoch 4/50
18/18 [==============================] - 14s 779ms/step - loss: 0.0368 - accuracy: 0.9910 - precision: 0.9964 - recall: 0.9857 - val_loss: 0.1406 - val_accuracy: 0.9493 - val_precision: 0.9079 - val_recall: 1.0000
Epoch 5/50
18/18 [==============================] - 14s 796ms/step - loss: 0.0601 - accuracy: 0.9892 - 

In [ ]:
# Model Evaluation
loss, accuracy, precision, recall = model.evaluate(val_data)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Validation Precision: {precision:.4f}")
print(f"Validation Recall: {recall:.4f}")

5/5 [==============================] - 3s 450ms/step - loss: 0.0179 - accuracy: 0.9928 - precision: 1.0000 - recall: 0.9855
Validation Loss: 0.0179
Validation Accuracy: 0.9928
Validation Precision: 1.0000
Validation Recall: 0.9855


In [ ]:
# Generate predictions for the validation dataset
val_preds = (model.predict(val_data) > 0.5).astype("int32")
true_labels = val_data.classes

# Classification report and confusion matrix
print("\nClassification Report:")
print(classification_report(true_labels, val_preds))
print("\nConfusion Matrix:")
print(confusion_matrix(true_labels, val_preds))

5/5 [==============================] - 3s 465ms/step

Classification Report:
              precision    recall  f1-score   support

           0       0.46      0.46      0.46        69
           1       0.46      0.46      0.46        69

    accuracy                           0.46       138
   macro avg       0.46      0.46      0.46       138
weighted avg       0.46      0.46      0.46       138


Confusion Matrix:
[[32 37]
 [37 32]]


In [12]:
print(model.input_names)
model.get_layer(index=0)._name = "mobilenetv2_input"
print(model.input_names)

['mobilenetv2_1.00_224_input']
['mobilenetv2_1.00_224_input']


In [13]:
# Save the model as TFLite
tflite_model_path = "/content/ktp_classifier2.tflite"

# Convert the model to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model to a file
with open(tflite_model_path, "wb") as f:
    f.write(tflite_model)

print(f"TFLite model saved at {tflite_model_path}")

TFLite model saved at /content/ktp_classifier.tflite


# TESTING

In [31]:
import os
import cv2
import tensorflow as tf
import numpy as np

tflite_model_path = "ktp_classifier2.tflite"

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Define image size and class labels
IMG_SIZE = (224, 224)
class_labels = {0: "KTP", 1: "Not KTP"}

def preprocess_image(image_path):
    """Preprocess the image to match the model input requirements."""
    img = cv2.imread(image_path)
    img_resized = cv2.resize(img, IMG_SIZE)
    img_normalized = img_resized / 255.0
    return np.expand_dims(img_normalized, axis=0).astype(np.float32)

def testWithFolder(test_folder,expectedLabel):
    for file_name in os.listdir(test_folder):
        file_path = os.path.join(test_folder, file_name)
        if not file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue

        input_data = preprocess_image(file_path)

        # inference
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details[0]['index'])

        # Get prediction and confidence score
        confidence = output_data[0][0]
        predicted_label = class_labels[1] if confidence > 0.5 else class_labels[0]
        correctLabel = f"[{'✓' if predicted_label == expectedLabel else '✗'}]"
        # Print result
        print(f"Image: {file_name}\n Prediction: {predicted_label} Should be {expectedLabel}\n{correctLabel}\n")

In [32]:
test_KTP = r"tesKTP"
test_NonKTP = r"tesNonKTP"

testWithFolder(test_KTP, "KTP")
testWithFolder(test_NonKTP, "Not KTP")

Image: download (1).jpg
 Prediction: Not KTP Should be KTP
[✗]

Image: download (2).jpg
 Prediction: Not KTP Should be KTP
[✗]

Image: download.jpg
 Prediction: KTP Should be KTP
[✓]

Image: images (1).jpg
 Prediction: Not KTP Should be KTP
[✗]

Image: images (2).jpg
 Prediction: KTP Should be KTP
[✓]

Image: images (3).jpg
 Prediction: KTP Should be KTP
[✓]

Image: images.jpg
 Prediction: KTP Should be KTP
[✓]

Image: download (1).jpg
 Prediction: Not KTP Should be Not KTP
[✓]

Image: download (2).jpg
 Prediction: Not KTP Should be Not KTP
[✓]

Image: download (3).jpg
 Prediction: Not KTP Should be Not KTP
[✓]

Image: download (4).jpg
 Prediction: Not KTP Should be Not KTP
[✓]

Image: download.jpg
 Prediction: Not KTP Should be Not KTP
[✓]

Image: images (1).jpg
 Prediction: Not KTP Should be Not KTP
[✓]

Image: images.jpg
 Prediction: Not KTP Should be Not KTP
[✓]

